**정형 데이터**<br>
엑셀 파일과 같이 정리된 형태로 되어 있는 데이터<br>
앙상블 학습을 주로 사용<br>
**비정형 데이터**<br>
정리된 형태로 표현하기 어려운 데이터, 글/그림/음악 등<br>
신경망 알고리즘을 주로 사용

**랜덤 포레스트**<br>
결정 트리를 랜덤하게 만들어 트리의 숲을 만듦<br>
1. 부트스트랩 샘플 생성
2. 노드 분할 시, 일부 특성을 무작위로 골라 그 중 최선의 분할을 찾음
 - RandomForestClassifier: $\sqrt{특성개수}$ 만큼의 특성 선택
 - RandomForestRegressor: 전체 특성 사용
3. 각 결정 트리의 예측을 사용해 최종 예측
 - 분류: 가장 높은 확률을 가진 클래스
 - 회귀: 각 트리의 예측 평균

**부트스트랩 샘플**<br>
데이터 세트에서 랜덤하게 뽑아서 샘플을 만드는데, 중복을 허용함

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


**랜덤 포레스트의 특성 중요도**<br>
각 결정 트리에서 나온 특성 중요도를 취합<br>
특성의 일부를 랜덤 선택하여 결정 트리를 훈련하므로, 더 많은 종류의 특성이 훈련에 기여할 수 있음<br>
과대적합 방지, 일반화 성능 상승<br>

In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


**OOB 샘플**<br>
부트스트랩 샘플에 포함되지 않고 남는 샘플<br>
이걸 사용하여 자체적으로 모델 평가

In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


**엑스트라 트리**<br>
랜덤 포레스트와 달리, 부트스트랩 샘플 미사용 & 노드 분할 시 무작위 분할

In [6]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


**그래디언트 부스팅**<br>
깊이가 얕은 결정 트리를 사용하여 경사 하강법을 수행<br>
일반적으로 과대적합에 강함 -> 높은 일반화 성능
일부 특성에 더 집중하는 경향이 있음

In [7]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) #n_estimators: 결정 트리 개수 (subsamples: 트리 훈련에 사용될 훈련 세트의 비율 정함, def=1.0)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
gb.fit(train_input, train_target)
print(gb.feature_importances_)

0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


**히스토그램 기반 그레이디언트 부스팅**<br>
1. 입력 특성을 256개의 구간으로 나눔 -> 최적의 분할 빠르게 찾기
2. 256개의 구간 중 1개를 떼서 누락된 값을 위해 사용

In [9]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


**permutation_importance**<br>
특성을 랜덤하게 섞어, 모델이 성능이 변화하는지 관찰하면서 어떤 특성이 중요한지 계산

In [10]:
hgb.fit(train_input,train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1) #train
print(result.importances_mean)
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1) #test
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


**XGBoost**<br>
다양한 부스팅 알고리즘 지원<br>
**LightGBM**<br>
히스토그램 기반 그레이디언트 부스팅 라이브러리

In [ ]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

In [ ]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))